# Image Classification Notebook
This notebook demonstrates the full pipeline of an image classification task using TensorFlow and Keras.

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


## Data Preprocessing
Here we perform the data preprocessing steps like scaling, augmentation, and splitting the data into training and validation sets.

In [2]:
# Set up directories for training and testing
train_data_dir = '/path/to/train_data'
test_data_dir = '/path/to/test_data'

# Data Preprocessing with ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


## Model Architecture
Next, we build the model architecture. This includes layers like `Dense`, `Flatten`, and `Input`.

In [3]:
# Build the model
model = Sequential()

# Using Input layer to avoid 'input_shape' warning
model.add(Input(shape=(64, 64, 3)))  # Adjust shape based on your dataset

# Add layers to the model
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Adjust the number of classes (10 in this case)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


## Training the Model
We now train the model using the training and validation data. We will also use early stopping to avoid overfitting.

In [4]:
# Set EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)


## Evaluating the Model
We evaluate the trained model on the validation data to check its performance.

In [5]:
# Evaluate the model
test_loss, test_acc = model.evaluate(validation_generator)
print('Test accuracy:', test_acc)


## Visualizing Results
We plot the accuracy and loss curves to visualize the training and validation performance over the epochs.

In [6]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


## Saving the Model
Finally, we save the trained model and its architecture to disk for future use.

In [7]:
# Save the model
model.save('image_classification_model.h5')

# Optionally, save the model architecture to JSON
model_json = model.to_json()
with open('image_classification_model.json', 'w') as json_file:
    json_file.write(model_json)


## Example Prediction
Here we show an example of how to load and use the model for making predictions on new images.

In [8]:
# Example of using the model for prediction (adjust based on your data)
# img_path = '/path/to/single/image.jpg'
# img = tf.keras.preprocessing.image.load_img(img_path, target_size=(64, 64))
# img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
# img_array = np.expand_dims(img_array, axis=0)
# prediction = model.predict(img_array)
# print(prediction)
